In [1]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/dan/miniconda3/envs/ml/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/bin/sh: line 1: nvidia-smi: command not found


🦥 Unsloth Zoo will now patch everything to make training faster!


/bin/sh: line 1: nvidia-smi: command not found
/bin/sh: line 1: nvidia-smi: command not found


# Load Model
- load pretrained model always
- if we already have a fine tune saved, can apply it
- if training model (not inference only), apply the training patch

In [2]:
max_seq_length = 8192
model_names = [
    'unsloth/Qwen2.5-0.5B-bnb-4bit',
    'unsloth/Qwen2.5-1.5B-bnb-4bit',
    'unsloth/Qwen2.5-3B-Instruct-unsloth-bnb-4bit',
]
model_name = model_names[2]
outpath = 'posttrained/qwen3b' # used for load/saving
print(f'Base Model: {model_name}')
load_in_4bit = True
dtype =  None  # none for autodetection

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

# some tokens that we might use later
IM_START = '<|im_start|>'
IM_END = '<|im_end|>'
EOS_TOKEN = tokenizer.eos_token

Base Model: unsloth/Qwen2.5-3B-Instruct-unsloth-bnb-4bit
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.546 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/bin/sh: line 1: nvidia-smi: command not found
/bin/sh: line 1: nvidia-smi: command not found


In [ ]:
# optional step if you are resuming from a saved lora
model.load_adapter(outpath)

In [11]:
# test inference, aka model output
from transformers import TextIteratorStreamer
from threading import Thread
import textwrap


def run_inference(model, tokenizer, input=None, max_tokens=256):
    # inference
    FastLanguageModel.for_inference(model)  # required toggle for unsloth model
    input_str = input or "First Citizen:"
    inputs = tokenizer(
        [input_str]*1,
        return_tensors="pt"
    ).to("cuda")

    text_streamer = TextIteratorStreamer(tokenizer)
    generation_kwargs = dict(
        inputs,
        streamer=text_streamer,
        max_new_tokens=max_tokens,
        use_cache=True,
    )
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    # unsloth printing for narrow view
    max_print_width = 100
    length = 0
    for i, new_text in enumerate(text_streamer):
        if i == 0:
            wrapped_text = textwrap.wrap(new_text, width=max_print_width)
            length = len(wrapped_text[-1])
            wrapped_text = "\n".join(wrapped_text)
            print(wrapped_text, end="")
        else:
            length += len(new_text)
            if length >= max_print_width:
                length = 0
                print()
            print(new_text, end="")

in1=f'''{IM_START}user
Continue the shakespearean dialogue below.
First Citizen:
Before we proceed any further, hear me speak.{IM_END}
{IM_START}assistant'''

# run_inference(model, tokenizer, input=in1)
# _ = model.train()  # switch back to training mode

# Training
- patch model for training
- load our data in
    - provided is a sample shakespeare dataset
    - the goal is to create a dataset with our Castle Agent (reward based on some accuracy metric)
- create trainer
- run trainer (wrapped with stats pre/post)

In [3]:
# explore datasets
from datasets import load_dataset
dataset = load_dataset('roneneldan/TinyStories', split="train[:2500]")
EOS_TOKEN = tokenizer.eos_token
def format_prompts_func(examples):
    return {'text': [i + EOS_TOKEN for i in examples['text']]}
dataset = dataset.map(format_prompts_func, batched=True)
split_data = dataset.train_test_split(test_size=0.1, seed=12)
train_dataset, val_dataset = split_data['train'], split_data['test']
print(len(train_dataset), len(val_dataset))
print(train_dataset[0])

2250 250
{'text': 'Once upon a time, there was a small girl named Susie. Susie loved to go exploring in the big backyard of her house. One day, Susie went outside with her teddy bear. As she was running around, she saw something hidden behind a bush. She went near it, and when she peeked behind the bush, she found a kitty! Susie was so excited to see the kitty that she ran over to it. \n\nSusie said to the kitty, "Hi kitty, why are you here all alone?" \n\nThe kitty meowed and then said, "I\'m lost. I don\'t know how to get back home." \n\nSusie felt so sorry for the kitty and she wanted to help him. She said, "Don\'t worry, kitty. I can help you find your way home! Let\'s go visit and see." \n\nSo Susie and the kitty started walking to find the kitty\'s home. After a while, they ran into some trouble. There was a big angry dog that blocked their way. Susie and the kitty were scared, but Susie kept her head and said, "Don\'t worry, kitty. We can find another way." So they both ran away

In [4]:
# patch the quantized model to allow for training
model = FastLanguageModel.get_peft_model(
    model,
    r=128, # 32, 64, 128
    lora_alpha=256, # double r
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj',
                    'gate_proj', 'up_proj', 'down_proj',

                    'embed_tokens', 'lm_head'],  # last two for "continual pretraining" of ood data?
    lora_dropout=0.,
    bias='none',
    use_gradient_checkpointing='unsloth',
    random_state=3407,
    use_rslora=True,  # "rank stablized" lora
    loftq_config=None,
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.2.15 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


In [5]:
# implement trainer
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field='text',
    max_seq_length=max_seq_length,
    dataset_num_proc=8, # i believe this is num gpus?
    args=UnslothTrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,

        # warmup_ratio=0.1,
        num_train_epochs=3,

        learning_rate=5e-5,
        embedding_learning_rate=5e-6,

        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        optim='adamw_8bit',
        weight_decay=0.0,
        lr_scheduler_type='cosine',
        seed=3407,

        # evaluation added
        evaluation_strategy='epoch',
        save_strategy='epoch',
        logging_steps=1,

        output_dir='outputs',
        report_to='none'
    )
)

/home/dan/miniconda3/envs/ml/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/dan/miniconda3/envs/ml/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Applying chat template to train dataset (num_proc=8): 100%|██████████| 2250/2250 [00:00<00:00, 2593.67 examples/s]
Truncating train dataset (num_proc=8): 100%|██████████| 2250/2250 [00:00<00:00, 16270.89 examples/s]
Applying chat template to eval dataset (num_proc=8): 100%|██████████| 250/250 [00:00<00:00, 303.36 examples/s]
Truncating eval dataset (num_proc=8): 100%|██████████| 250/250 [00:00<00:00, 2233.94 examples/s]


In [6]:
# BEFORE RUN: gpu stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / (1024**3), 3)
max_memory = round(gpu_stats.total_memory/(1024**3),3)
print(f'GPU = {gpu_stats.name}. Max memory = {max_memory} GB.')
print(f'{start_gpu_memory} GB of memory reserved.')

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.546 GB.
3.744 GB of memory reserved.


In [8]:
# do the training
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,250 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 420
 "-____-"     Number of trainable parameters = 861,798,400
/bin/sh: line 1: nvidia-smi: command not found


Epoch,Training Loss,Validation Loss
0,1.518800,1.382645
1,0.878200,1.445567
2,0.317300,1.826757


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [9]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")


658.1967 seconds used for training.
10.97 minutes used for training.
Peak reserved memory = 10.049 GB.
Peak reserved memory for training = 6.305 GB.
Peak reserved memory % of max memory = 42.678 %.
Peak reserved memory for training % of max memory = 26.777 %.


In [12]:
# run some inference
input_text = '''<|im_start|>user Continue the shakespearean dialogue below.
First Citizen: What, sayeth I, have you done!?<|im_end|>
<|im_start|>assistant'''
run_inference(model, tokenizer, input=input_text)
_ = model.train()

<|im_start|>user Continue the shakespearean dialogue below. First Citizen: What, sayeth I, have you
done!?<|im_end|>
<|im_start|>assistant: I have made a cake! It is so yummy and sweet. I am very proud of it. 
First Citizen: That's great! 
What a brave thing to do! Do you want me to help you display it? 
<|im_end|>